In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from matplotlib.ticker import MaxNLocator


In [29]:
# import mc data file
# mcfile = 'mc_data/mc_data_roots100.npy'
mcfile = '../dsa_mc/testing_new.npy'
mcdata = np.load(mcfile)
mc_df = pd.DataFrame(mcdata, columns=['s', 'Q', 'x', 'delta', 'pT', 'z', 'y', 'phi_kp', 'phi_Dp', 
                                      'num_dsa', 'den_dsa', '<1>', '<cos(phi_kp)>', '<cos(phi_Dp)>', 
                                     '<cos(phi_Dp)cos(phi_kp)>', '<sin(phi_Dp)sin(phi_kp)>'])

mc_df['t'] = np.abs(mc_df['delta'])**2

# mcfile2 = 'mc_data/mc_data_roots70.npy'
# # mcfile = 'mc_data/testing.npy'
# mcdata2 = np.load(mcfile2)
# mc_df2 = pd.DataFrame(mcdata2, columns=['s', 'Q', 'x', 'delta', 'pT', 'z', 'y', 'phi_kp', 'phi_Dp', 
#                                       'num_dsa', 'den_dsa', '<1>', '<cos(phi_kp)>', '<cos(phi_Dp)>', 
#                                      '<cos(phi_Dp)cos(phi_kp)>', '<sin(phi_Dp)sin(phi_kp)>'])
# mc_df2['t'] = np.abs(mc_df2['delta'])



In [31]:
mc_df.tail()

,s,Q,x,delta,pT,z,y,phi_kp,phi_Dp,num_dsa,den_dsa,<1>,<cos(phi_kp)>,<cos(phi_Dp)>,<cos(phi_Dp)cos(phi_kp)>,<sin(phi_Dp)sin(phi_kp)>,t
9995,10000.0,6.335892,0.005499,0.906515,7.689941,0.501991,0.730074,3.036585,4.935072,0.050661,0.458657,0.157197,0.000097,0.000219,0.000252,-0.003253,0.821769
9996,10000.0,8.407755,0.009091,1.525762,7.882087,0.229061,0.777569,5.877347,1.960568,0.027966,0.129874,0.191108,-0.009921,-0.051327,-0.001855,-0.008805,2.327950
9997,10000.0,6.684007,0.008085,1.660686,8.555261,0.350040,0.552561,5.387314,3.121252,0.026117,0.104007,0.159740,-0.010708,-0.031044,0.004141,-0.008388,2.757879
9998,10000.0,7.510529,0.007650,0.325753,7.365236,0.674162,0.737373,3.013250,3.014324,0.028013,0.298669,0.154046,0.004822,0.006049,-0.000642,-0.001287,0.106115
9999,10000.0,6.171778,0.009633,0.895415,7.522116,0.441679,0.395422,3.740937,4.134584,0.027339,0.256310,0.123433,-0.002979,-0.005514,0.001233,-0.003345,0.801768


In [43]:
# function to plot histograms

# setup plotting
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 12
plt.rcParams["legend.fontsize"] = 12
plt.rcParams["axes.labelsize"] = 14  
plt.rcParams["xtick.labelsize"] = 12  
plt.rcParams["ytick.labelsize"] = 12
plt.rcParams["axes.titlesize"] = 16  
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{amsmath}"
%config InlineBackend.figure_format = 'retina'

labels = {
         'num_dsa': 'DSA',
         'den_dsa': 'Total',
         '<1>': r'$A_{LL}$',
         '<cos(phi_kp)>': r'$\langle \cos(\phi_{kp}) \rangle$',
         '<cos(phi_Dp)>': r'$\langle \cos(\phi_{\Delta p}) \rangle$',
         '<cos(phi_Dp)cos(phi_kp)>': r'$\langle \cos(\phi_{kp}) \cos(\phi_{\Delta p}) \rangle$',
         '<sin(phi_Dp)sin(phi_kp)>': r'$\langle \sin(\phi_{kp}) \sin(\phi_{\Delta p}) \rangle$'
         }

def plot_histogram(dfs, plot_var, weights, constraints={}, **options):

    asp_ratio = 6/3
    psize = 4

    fig, axs = plt.subplots(1, 2, figsize=(asp_ratio*psize, psize), sharex=True, gridspec_kw={'width_ratios': [5, 1]})
        
    colors = ['black', 'red', 'blue', 'green']
    linestyles = ['-', '--', '-.']

    lumi = options.get('lumi', 10) # total integrated luminosity in fb^-1
    lumi *= options.get('efficiency', 1) # correct for detector efficiency

    for idf, df in enumerate(dfs):
        
        # make bins
        range = [df[plot_var].min(), df[plot_var].max()]
        nbins = options.get('nbins', 10)
        bin_width = options.get('binwidth', (range[1] - range[0])/nbins)
        bins = options.get('bins', np.arange(np.floor(range[0]/bin_width)*bin_width, np.ceil(range[1]/bin_width)*bin_width, bin_width))

        # enforce constraints
        mask = pd.Series(True, index=df.index)
        for var, (low, high) in constraints.items(): mask &= df[var].between(low, high)
        fixed_df = df[mask]

        if fixed_df.empty:
            print('Error: selected dataframe is empty - constraints are too strict')
            return
    
        for iw in weights: assert iw in list(df.columns), f'Error: option for weight {iw} not recognized'
        
        # make plot data         
        for iw, weight in enumerate(weights):
    
            total_counts, plot_bins = np.histogram(fixed_df[plot_var], bins=bins, weights=fixed_df['den_dsa'])
            plot_counts, _ = np.histogram(fixed_df[plot_var], bins=bins, weights=fixed_df[weight])
            bin_centers = 0.5*(plot_bins[:-1]+plot_bins[1:])
    
            # ensure bins are properly averaged
            n_entries, _ = np.histogram(fixed_df[plot_var], bins=bins)
            plot_counts = np.array([icount/(bin_width*ientry) if ientry != 0 else 0 for icount, ientry in zip(plot_counts, n_entries)])
            total_counts = np.array([icount/(bin_width*ientry) if ientry != 0 else 0 for icount, ientry in zip(total_counts, n_entries)])
      
            # errors are calculated for a given integrated luminosity (5% systematic error added per 1505.05783)
            if '<' in weight:
                stat_errors = np.array([np.sqrt((1+ic)/(lumi*tc)) if tc != 0 else 0 for ic, tc in zip(plot_counts, total_counts)])
            else:
                stat_errors = np.sqrt(total_counts/lumi)
            sys_errors = 0.05*np.abs(plot_counts)  
            total_errors = np.sqrt((stat_errors**2)+(sys_errors)**2)

            if len(dfs) > 1: plot_label = labels[weight] + f'({idf})'
            else: plot_label = labels[weight] 
    
            # make data plot
            axs[0].errorbar(
                bin_centers, plot_counts, yerr=total_errors,
                fmt=options.get('fmt', 'o'), 
                capsize=3, elinewidth=0.5, capthick=0.5, color=colors[iw + idf],
                label=plot_label,
            )

            axs[0].step(bin_centers, plot_counts, where='mid', linestyle=linestyles[idf], color=colors[iw+idf], linewidth=1)
    
        axs[0].legend(frameon=False)

        # make info box for constraints
        info_text = fr'Integrated luminosity: ${lumi}\,\, \mathrm{{fb}}^{{-1}}$'+'\n'
        info_text += fr'$\sqrt{{s}} = {round(np.sqrt(df['s'][0]))}\,\, \mathrm{{GeV}}$'+'\n'
        info_text += '\nCuts:'
        for var, (low, high) in constraints.items():
            if var == 'Q' or var == 'pT' or var == 'delta':
                info_text += '\n'+ fr'$ {low} < {var}\,[\mathrm{{GeV}}] < {high}$'
            elif var == 't':
                info_text += '\n'+ fr'$ {low} < {var}\,[\mathrm{{GeV}}^2] < {high}$'
            else:
                info_text += '\n' + fr'$ {low} < {var} < {high}$'
        axs[1].text(
            0.0, 0.0, info_text, 
            ha='left', va='bottom', 
            fontsize=12, wrap=True, 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.3)
        )
        axs[1].set_axis_off()

        # set plot info
        if options.get('y_limits'): axs[0].set_ylim(options.get('y_limits'))
        axs[0].set_xlim(options.get('x_limits', [plot_bins[0], plot_bins[-1]]))
        axs[0].set_title(options.get('title', ''))
        axs[0].set_ylabel(options.get('y_label', 'Frequency'), loc='top')
        axs[0].set_xlabel(options.get('x_label', fr'${plot_var}$'), loc='right')
        axs[0].set_yscale(options.get('y_scale', 'linear'))
        axs[0].set_xscale(options.get('x_scale', 'linear'))
        axs[0].tick_params(axis="both", direction="in", length=5, width=1, which='both', right=True, top=True)
        axs[0].grid(options.get('grid', False)) 
        axs[0].xaxis.set_major_locator(MaxNLocator(nbins=10))
        if options.get('zero_line', False): axs[0].axhline(y=0, color='gray', linestyle='--', linewidth=1)
        if options.get('min_pT_line', False) and plot_var == 'pT': 
            axs[0].axvline(x=1, color='gray', linestyle='--')
            axs[0].fill_betweenx(axs[0].get_ylim(), axs[0].get_xlim()[0], 1, color='gray', alpha=0.25)
        plt.tight_layout()
        if options.get('saveas'): 
            if 'DNS' in options.get('saveas'): 
                pass
            else:
                fig.savefig(options.get('saveas'), dpi=400, bbox_inches="tight")




In [47]:
# plot relevant quantities 

Q_bins = np.linspace(6, 10.0, num=25)
pT_bins = np.linspace(0, 15.0, num=25)
phi_bins = [i/(2*np.pi) for i in range(15)]


lumi = 1000000
Q_min = 6.0 
Q_max = 9.5
y_min = 0.4
y_max = 0.5
z_min = 0.2
z_max = 0.5
x_min = 0.008
x_max = 0.01

tag = 'test'

plot_histogram([mc_df], 'pT', ['den_dsa', 'num_dsa'], 
                   {
                    # 'Q': [Q_min, Q_max],
                    # 'y': [y_min, y_max],
                    # 'z': [z_min, z_max],
                   }, 
                   lumi = lumi,
                   bins = pT_bins, 
                   y_scale='log', 
                   # x_scale='log',
                   # x_limits = [0, 11], 
                   nbins = 20, 
                   # y_limits = [10,10**9],
                   # zero_line = True,
                   min_pT_line = True,
                   y_label=r'$d\sigma /dp_T \,\,[fb*\mathrm{GeV}^{-1}]$', 
                   x_label=r'$p_T\, [\mathrm{GeV}]$',
                   saveas=f'plots/dsa_pT_{tag}.pdf'
              )



plot_histogram([mc_df], 'pT', ['<cos(phi_Dp)cos(phi_kp)>', '<sin(phi_Dp)sin(phi_kp)>','<cos(phi_Dp)>'], 
                   {
                    # 'Q': [Q_min, Q_max],
                    # 'pT': [2.5, 3.5]
                    # 'y': [y_min, y_max],
                    'z': [z_min, z_max],
                    # 'x': [x_min, x_max],
                   }, 
                   lumi = lumi,
                   efficiency = 1, 
                   # y_scale='log', 
                   # x_scale='log',
                   # x_limits = [5, 11], 
                   bins = pT_bins,
                   nbins = 20, 
                   y_limits = [-0.02,0.02],
                   zero_line = True,
                   min_pT_line = True,
                   y_label=r'$d \langle f(\phi_{kp}, \phi_{\Delta p}) \rangle /dp_T \,\,[\mathrm{GeV}^{-1}]$', 
                   x_label=r'$p_T\, [\mathrm{GeV}]$',
                   saveas = f'plots/LT_corr_{tag}.pdf'
                  )


plot_histogram([mc_df], 'pT', ['<1>', '<cos(phi_Dp)cos(phi_kp)>', '<sin(phi_Dp)sin(phi_kp)>','<cos(phi_Dp)>'], 
                   {
                    'Q': [Q_min, Q_max],
                    # 'pT': [2.5, 3.5]
                    # 'y': [y_min, y_max],
                    'z': [z_min, z_max],
                    'x': [x_min, x_max],
                   }, 
                   lumi = lumi,
                   efficiency = 1, 
                   # y_scale='log', 
                   # x_scale='log',
                   # x_limits = [5, 11], 
                   bins = pT_bins,
                   nbins = 20, 
                   y_limits = [-0.1,0.65],
                   zero_line = True,
                   min_pT_line = True,
                   y_label=r'$d \langle f(\phi_{kp}, \phi_{\Delta p}) \rangle /dp_T \,\,[\mathrm{GeV}^{-1}]$', 
                   x_label=r'$p_T\, [\mathrm{GeV}]$',
                   saveas = f'plots/all_corr_{tag}.pdf'
                  )


